# Tree representation

有個比較簡易的方式是使用python list來實作，核心概念就是

[ root content, left child, right child ] 所以

[ 'a',
  [ 'b', 'c' 'd'],
  [ 'e', 'f']
]

會變成

           a
        /    \
       b      e
     /   \    /
    c     d  f
 
 
正常狀況應該是使用node結構來定義tree拉，只是我們先來看看簡單的表示方式



In [23]:
def BinaryTree(r):
    return [r, [], []]

def insertLeft(root, newBranch):
    origin = root.pop(1)
    if len(origin) > 1:
        root.insert(1, [newBranch, origin, []])
    else:
        root.insert(1, [newBranch, [], []])
        
def insertRight(root, newBranch):
    origin = root.pop(2)
    if len(origin) > 1:
        root.insert(1, [newBranch, [], origin])
    else:
        root.insert(1, [newBranch, [], []])
        
def getRootVal():
    return root[0]

def setRootVal(root,newVal):
    root[0] = newVal

def getLeftChild(root):
    return root[1]

def getRightChild(root):
    return root[2]

這個實現並沒有說非常嚴謹，看看insertLeft，他都是固定把原本的branch放到新插入的branch的左邊，印象我以前學的時候，照理說是要和原branch比較後，再決定是接在新branch的左邊或右邊，另外種實現方式就是自定義node結構，物件導向設計，

In [24]:
class TNode:
    
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None
        

class BinaryTree:
    
    def __init__(self, root_value=0):
        self._root = TNode(root_value)
        
    def insertLeft(self, newBranch):
        if self._root.left is None:
            # nothing to consider
            self._root.left = newBranch
        else:
            newBranch.left = self._root.left
            self._root = newBranch
    
    def insertRight(self, newBranch):
        if self._root.right is None:
            self._root.right = newBranch
        else:
            newBranch.right = self._root.right
            self._root = newBranch
            
    @property
    def root(self):
        return self._root
    
    @root.setter
    def root(self, value):
        self._root = TNode(value)
        
    @property
    def leftChild(self):
        return self._root.left
    
    @property
    def rightChild(self):
        return self._root.right
    
    

In [25]:
t = BinaryTree(0)
otherT = BinaryTree(2)

t.insertLeft(otherT)
t.leftChild.root.value

2

上面其實只是一個概念而已並非完整的樹的實作，詳細地必須看規範然去實作出來～


### Priority Queues

其實就是queues的一種變形，權重越高的點會在越前面，權重越低的當然就在越後面，課程中是說這邊會用data structure binary heap來實作


#### binary heap

有分min heap, max heap，依定義來區分就是parent node的值大於等於child值的話就是max heap，反之則是min heap

heap data structure就是一種樹的資料結構

binary heap，則是一個更加嚴謹定義的heap data structure，binary heap想當然爾就是從binary tree來的，而且是**fully complete binary tree**，也就是說每個subtree都有兩個child，除了最後一個subtree可以有單個，詳細定義再去看[wiki](https://en.wikipedia.org/wiki/Binary_heap)。


#### binary heap represent with array

畫個圖，就會發現要怎麼使用array對應，跟tree不一樣，不用用那麼多層，因為binary的關係，所以格式會比較固定，因此在array上的安排我們只要由上往下，從左到右一個一個將元素丟入陣列即可。

接著你會發現幾個特點 

前提是index從一開始
node[i]其

 - parent會是 i//2
 - left child會是 2i
 - right child會是 2i+1




In [26]:
class BinHeap:
    
    def __init__(self):
        self.heapLst = [0] # 這裡會先插入一個0其實是為了之後array index取得left child, right child方便
        self.currentSize = 0
        
    def percUp(self, i):
        
        while i // 2 > 0:
            
            if self.heapLst[i] < self.heapLst[i//2]:
                # do swap,為了符合heap規範 the root value of subtree must be bigger than child
                self.heapLst[i], self.heapLst[i//2] = self.heapLst[i//2], self.heapLst[i]
                
            i = i // 2
            
    def insert(self, k):
        # 邏輯是這樣的，將元素append到最後一個，然後進行所謂的percolate up or heapify up whatever you like..
        # 這個動作就是要調整你新插入的元素位置，讓這個heap仍然遵守著binary heap的規範
        # 個人猜想是因為binary heap並沒有規範left child and right child大小順序，所以變成無法先搜尋該元素要插入的位置在進行插入，
        # 如果隨便往左或往右會造成樹不是binary，因此這個演算法大概是目前最好的方式了
        # 其效率是 lg n
        self.heapLst.append(k)
        self.currentSize += 1
        self.percUp(self.currentSize)
        
    def minChild(self, i):
        # 這個界線有點疑慮，已經有先假設不會帶入 i*2 > self.currentSize
        # 的先前條件了
        if i * 2 + 1 > self.currentSize:
            return i * 2
        else:
            if self.heapLst[i*2] > self.heapLst[i*2+1]:
                return i * 2 + 1
            else:
                return i * 2
        
    def percDown(self, i):
        # 跟percUp進行相反的邏輯，將元素從最上方移到最下面
        
        while i * 2 <= self.currentSize:
            
            mc = self.minChild(i)
            
            if self.heapLst[i] > self.heapLst[mc]:
                # do swap
                self.heapLst[i], self.heapLst[mc] = self.heapLst[mc], self.heapLst[i]
                
            i = mc
    
    def delMin(self):
        # 一樣為了容易維持binary heap的特性，這裡刪除的邏輯是，用最後一個元素來替換root，之後再進行調整樹結構(percolate down)
        min = self.heapLst[1]
        self.heapLst[1] = self.heapLst[self.currentSize]
        self.pop()
        self.percDown(1)
        return min
    
    def buildHeap(self, lst):
        self.currentSize = len(lst)
        self.heapLst = [0] + lst[:]
        
        i = len(lst) // 2
        while i > 0:
            self.percDown(i)
            i = i -1

#### Binary Search Tree

基於Binary Tree多加了幾個限制，就是left child的值會比parent node小，right child的值會比parent node大

In [27]:
class TreeNode:
    
    """實作樹可能會相較於cplus方便點，因為不需要處理pointer釋放等相關問題
    """
    
    def __init__(self, key, value, left=None, right=None, parent=None):
        self.key = key
        self.value = value
        self.right = right
        self.left = left
        self.parent = parent
        
    def isLeftChild(self):
        return self.parent and self.parent.left == self
    
    def isRightChild(self):
        return self.parent and self.parent.right == self
    
    def hasLeftChild(self):
        return self.left
    
    def hasRightChild(self):
        return self.right
    
    def isRoot(self):
        return not self.parent
    
    def isLeaf(self):
        return not (self.left or self.right)
    
    def hasAnyChildren(self):
        return self.left or self.right
    
    def hasBothChildren(self):
        return self.left and self.right
    
    # 下面這兩個函式，我個人沒去用，我是用自己的邏輯來實作這棵樹 
    
    def spliceOut(self):
    # 意義不明，是在被刪除的節點，具有兩個children的情況，才會用的
        pass
    
    def replaceNodeData(self, key, value, lChild, rChild):
        self.key = key
        self.value = value
        self.left = lChild
        self.right = rChild
        # 這邊行為意義不明，上面都assign left and right了，怎麼可能會沒有left, right..
        if self.hasLeftChild():
            self.left.parent = self
            
        if self.hasRightChild():
            self.right.parent = self
            
    # 
            
    def replace_node_in_parent(self, new_node=None):
        # 這是在將某點移除時，一定會做的動作，將舊節點拿掉，換成新節點
        if self.parent:
            if self == self.parent.left:
                self.parent.left = new_node
            else:
                self.parent.right = new_node
                
        if new_node:
            new_node.parent = self.parent
            
    def findMin(self):
        cur = self
        
        while cur.left:
            cur = cur.left
                
        return cur
    
    def findMax(self):
        cur = self
        
        while cur.right:
            cur = cur.right
                
        return cur
    
    def findSuccessor(self):
        # wiki 上面的 binary search tree定義是這樣的
        # a node's in-order successor is its right subtree's left-most child, 
        # and a node's in-order predecessor is the left subtree's right-most child
        # 其實就是一棵樹以中序表示，其successor就會是右子樹的最小值，predecessor就會是左子樹的最大值，
        # 會這樣命名大概是因為，中序表示是這樣的，1. 取值or do something with node 2. 往左 3. 往右
        # 所以左子樹才會被稱為 predecessor，右子樹被稱為 successor 
        if self.hasRightChild():
            return self.right.findMin()
            
        return None
        
    def findPredecessor(self):
        if self.hasLeftChild():
            return self.left.findMax()
        
        return None
            
class BinarySearchTree:
    
    def __init__(self):
        self.root = None
        self.size = 0
        
    def __len__(self):
        return self.size
    
    def put(self, key, value):
        if self.root:
            self._put(key, value, self.root)
        else:
            self.root = TreeNode(key, value)
        
        self.size += 1
            
    def _put(self, key, value, currentNode):
        if key < currentNode.key:
            if currentNode.hasLeftChild():
                self._put(key, value, currentNode.left)
            else:
                currentNode.left = TreeNode(key, value, parent=currentNode)
        else:
            if currentNode.hasRightChild():
                self._put(key, value, currentNode.right)
            else:
                currentNode.right = TreeNode(key, value, parent=currentNode)
        
    def callback(self):
        res = []
        
        def inner(node):
            res.append(str(node.key))
            
        return res, inner
        
        
    def inorder(self, node, callback):
        if node:
            self.inorder(node.left, callback)
            callback(node)
            self.inorder(node.right, callback)
        
    def __str__(self):
        res, callback = self.callback()
        self.inorder(self.root, callback)
        
        return ' '.join(res)
    
    def __setitem__(self, key, value):
        self.put(key, value)
        
    def get(self, key):
        if self.root:
            res = self._get(key, self.root)
            
            if res:
                return res.value
            
        return None
    
    def _get(self, key, currentNode):
        if not currentNode:
            return None
        if key == currentNode.key:
            return currentNode
        elif key < currentNode.key:
            return self._get(key, currentNode.left)
        else:
            return self._get(key, currentNode.right)
    
    def __getitem__(self, key):
        return self.get(key)
    
    def __contaion__(self, key):
        if self._get(key, self.root):
            return True
        
        return False
    
    def delete(self, key):
        node = self._get(key, self.root)
        if node:
            self.remove(node)
            self.size = self.size - 1
        else:
            raise KeyError('key not in the tree')
    
    def __delitem__(self, key):
        self.delete(key)
    
    def remove(self, node):
        # 情況分為三種
        # 1. 被刪除的節點 本身為 leaf node 
        # 2. 被刪除的節點 本身具有一個 child
        # 3. 被刪除的節點 本身具有兩個 children
        # 1和2 都是比較簡單的處理，只有第三點，需要考量一下
        
        if node.isLeaf():
            node.replace_node_in_parent(None)
        elif node.hasBothChildren():
            # 當有兩個children時，其實就是第一先尋找要拿來替代的節點候選人
            # 再來就是將原本node的key, value換成candidate
            # 最後就是remove掉candidate node
            candidate = node.findSuccessor() or node.findPredecessor()
            node.key = candidate.key
            node.value = candidate.value
            self.remove(candidate)
        else:
            if node.hasLeftChild():
                node.replace_node_in_parent(node.left)
            else:
                node.replace_node_in_parent(node.right)
    
    

In [28]:
mytree = BinarySearchTree()
mytree[3]="red"
mytree[4]="blue"
mytree[10]="yes"
mytree[11]='ok'
mytree[12]='shit'
mytree[5]="ha"
mytree[6]="yellow"
mytree[2]="at"

print(mytree[6])
print(mytree[2])
print(mytree)

mytree.delete(10)
print(mytree)

yellow
at
2 3 4 5 6 10 11 12
2 3 4 5 6 11 12


從這裡就可以看出前面為啥會叫做predecessor和successor，這裡我用中序印出樹，然後去刪除節點10，還記得前面說什麼嗎？ 找出 successor 來替代，找法是找右子樹的最小值，從我上面印出來的數字， `11 12` 確確實實是在10之後，所以才會被稱為successor，這邊是我個人猜測命名的原因拉

# Tree Problems

Given a binary tree, check whether it’s a binary search tree or not.
Again, no solution cell, just worry about your code making sense logically. Hint: Think about tree traversals.

所以看起來這邊我要定義自己的樹結構！ 感覺也是滿有可能的拉，就是面試想要看看你的想法

In [29]:
class TreeNode:
    
    def __init__(self, val):
        self.left = None
        self.right = None
        self.val = val
        
    def __str__(self):
        return f"node {self.val}"
    
    def __repr__(self):
        return f"<TreeNode({self.val})>"
        
        
def solution(root: TreeNode) -> bool:
    if not root:
        return True
    
    if root.left:
        if root.left.val >= root.val:
            return False
        
    if root.right:
        if root.right.val <= root.val:
            return False
    
    
    return solution(root.left) and solution(root.right)

# ... 這個解法，並非完全正確！ 比如： 要是左子樹的右節點大於tree's root 那就會錯了！
# 比較簡單的解法是用inorder的方式！ 然後去看看sorted後的順序是否相同，一樣的話那就是了

def inorder_keys(root: TreeNode) -> list:
    if root is None:
        return [] 
    return inorder_keys(root.left) + [root.val] + inorder_keys(root.right)

def real_solution(root: TreeNode) -> bool:
    tree = inorder_keys(root)
    return tree == sorted(tree)

In [30]:
def establish_tree(nodes):
    for index in range(len(nodes)//2):
            nodes[index].left = nodes[2*index+1]
            if 2*index+2 < len(nodes):
                nodes[index].right = nodes[2*index+2]

def inorder_print(node):
    if not node:
        return 
    
    inorder_print(node.left)
    print(node.val)
    inorder_print(node.right)



In [31]:
import unittest

class BinaryTreeTest(unittest.TestCase):
    
    def check_false_binary_tree(self):
        nodes = [TreeNode(num) for num in range(1, 7)]
        establish_tree(nodes)
        self.assertFalse(real_solution(nodes[0]))
        print("song!")
        
    def check_true_binary_tree(self):
        binary_nodes = [TreeNode(5), TreeNode(4), TreeNode(6), TreeNode(3), TreeNode(7)]
        establish_tree(binary_nodes)
        self.assertFalse(real_solution(binary_nodes[0]))
        print("song!")
        
test = BinaryTreeTest()
test.check_false_binary_tree()
test.check_true_binary_tree()

song!
song!


# Tree Level Order Print
Given a binary tree of integers, print it in level order. The output will contain space between the numbers in the same level, and new line between different levels. For example, if the tree is:


The output should be:
```
1 
2 3 
4 5 6
```

In [32]:
from collections import deque

def levelOrderPrint(tree: TreeNode):
   
    if not tree:
        return
    
    q = deque([tree])
    
    current_count, next_count = 1, 0
  
    while len(q):
        node = q.popleft()
        if current_count == 0:
            print(node.val, end='')
        else:
            print(node.val, end=' ')
            
        current_count -= 1
        
        if node.left:
            q.append(node.left)
            next_count += 1
        
        if node.right:
            q.append(node.right)
            next_count += 1
            
        if current_count == 0:
            print()
            current_count, next_count = next_count, 0
       
            

In [33]:

class LevelOrderPrintTest(unittest.TestCase):
    
    def test_level_order_print(self):
        nodes = [TreeNode(num) for num in range(1, 7)]
        establish_tree(nodes)
        levelOrderPrint(nodes[0])
        
        
t = LevelOrderPrintTest()
t.test_level_order_print()

1 
2 3 
4 5 6 


# Trim a Binary Search Tree

Given the root of a binary search tree and 2 numbers min and max, trim the tree such that all the numbers in the new tree are between min and max (inclusive). The resulting tree should still be a valid binary search tree. So, if we get this tree as input:

In [47]:
# I provide a non inplace way
def trim_binary_search_tree(root: TreeNode, minimum: int, maximum: int) -> TreeNode:
    if not root:
        return root
    
    new_root = TreeNode(root.val)
    
    if root.left and root.left.val >= minimum:
        new_root.left = trim_binary_search_tree(root.left, minimum, maximum)
    if root.right and root.right.val <= maximum:
        new_root.right = trim_binary_search_tree(root.right, minimum, maximum)
        
    
    return new_root

# However, I think the way provided by the course is more elegant.
def trim_binary_search_tree_course(root: TreeNode, minimum: int, maximum: int) -> TreeNode:
    if not root:
        return root
    
    root.left = trim_binary_search_tree_course(root.left, minimum, maximum)
    root.right = trim_binary_search_tree_course(root.right, minimum, maximum)
    
    if minimum <= root.val <= maximum:
        return root
    
    if root.val > maximum:
        return root.left
    
    if root.val < minimum:
        return root.right
    

In [44]:
class TrimBinarySearchTreeTest(unittest.TestCase):
    
    def test_binary_test_search(self):
        binary_nodes = [TreeNode(5), TreeNode(4), TreeNode(6), TreeNode(3), TreeNode(7)]
        establish_tree(binary_nodes)
        levelOrderPrint(binary_nodes[0])
        trim_tree = trim_binary_search_tree(binary_nodes[0], 4, 6)
        
        levelOrderPrint(trim_tree)
        
t = TrimBinarySearchTreeTest()
t.test_binary_test_search()

5 
4 6 
3 7 
5 
4 6 
